# Explore RNAseq of zebrafish photoreceptors including developmental date
### [Transcription factors underlying photoreceptor diversity, Angueyra et al. eLife 2023;12:e81579](https://doi.org/10.7554/eLife.81579)

This notebook can be accessed here:  
[angueyraLab.github.io/drRNAseq/lab](angueyranih.github.io/drRNAseq/lab)  

## Follow instructions below to explore dataset openly available at: https://github.com/angueyraLab/drRNAseq.  
AND  
## Make comparisons with published RNAseq datasets containing zebrafish adult photoreceptors, including:  
- [Ogawa and Corbo (2021)](https://doi.org/10.1038/s41598-021-96837-z): FACS + 10x from adult rods, cones and bipolar cells  
- [Hoang _et al._ (2020)](https://doi.org/10.1126/science.abb8598): 10x from whole retina (extracted data from developing and adult photoreceptors and reclustered)  
- [Sun, Galicia and Stenkamp (2018)](https://doi.org/10.1186/s12864-018-4499-y): FACS + bulk RNAseq from adult rods (GFP+) and other retinal cells (GFP-)  
----
----

### How to use the notebook
- To explore the datasets, you will need to run the code in the cells below.  
- To run a cell, you need to click unto it to select it and press "Ctrl"+"Enter" or the small *play* symbol in the navigation bar on top of this notebook.  
- Cells that are currently running display [\*] on the left.  
- Cells that have finished running will display a number instead (e.g.[8])
---
---
#### Saving plots:
- Right click on the image and click on "Create New View for Output"
- Right click on image in Output View panel and click on "Save As..."

#### First: load python environment

In [ ]:
import pyodide_kernel
print("Successfully loaded pyolite version {0}".format(pyodide_kernel.__version__))

#### Second: load all necessary extensions (this may take a few minutes...)

In [ ]:
print("Loading extensions...")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from scipy.stats import zscore
import os
import pyodide
import piplite

# Load plotting function
URL = 'https://raw.githubusercontent.com/angueyraLab/drRNAseq/main/content/juanPlot-0a3-py2.py3-none-any.whl'
await piplite.install(URL)
from juanPlot import *
print("Successfully loaded extensions!")


# load datasets
print("Loading datasets...")
gf = pd.read_csv('./data/Angueyra2021_Photoreceptors.csv')
print("\t Angueyra et al. (2022)")

zfO = pd.read_csv('./data/Ogawa2021_10x_photoreceptors.csv')
print("\t Ogawa and Corbo (2021)")

zfH = pd.read_csv('./data/Hoang2020_10x_photoreceptors.csv')
print("\t Hoang et al. (2020)")

zfS = pd.read_csv('./data/Sun2018_FACS_Rods.csv')
print("\t Sun, Galicia, Stenkamp (2018)")

print("Successfully loaded datasets!\n\n")


#### Define some plotting bases and colors

In [ ]:
# plotStyle = 'Light'
plotStyle = 'Dark'

heatmapNormalization = True
heatmapNormalization = False

# photoreceptor Colors
pC = {
    'r' : '#747474', # Rods
    'u' : '#B540B7', # UV cones
    's' : '#4669F2', # S cones
    'm' : '#04CD22', # M cones
    'l' : '#CC2C2A', # L cones
    'm4': '#cdcd04', # opn1mws4-expressing M cones
    'onBC': '#ccf2ff', # on bipolar cells
    'offBC': '#663d00', # off bipolar cells
    'plt' : '',
    'zfO': '',
}

# default parameters for plotting
if plotStyle=='Dark':
    # dark background
    plt.style.use('dark_background')
    params = {"ytick.color" : "w", "xtick.color" : "w",
              "axes.labelcolor" : "w", "axes.edgecolor" : "w",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3, "ytick.major.width" : 3,
             "xtick.major.size" : 8, "ytick.major.size" : 8,
             "text.color" : "w"}
    plt.rcParams.update(params)
    baseColor = '#ffffff' #white
else:
    # white background
    plt.style.use('default')
    params = {"ytick.color" : "k", "xtick.color" : "k",
              "axes.labelcolor" : "k", "axes.edgecolor" : "k",
             "axes.linewidth" : 3,
             "xtick.major.width" : 3, "ytick.major.width" : 3,
             "xtick.major.size" : 8, "ytick.major.size" : 8,
             "text.color" : "k"}
    plt.rcParams.update(params)
    baseColor = '#000000' #black
    


print("Plotting style is: {0}".format(plotStyle))

***
# Index <a id='Index'></a>
- [Bar plot for single gene: Angueyra et al. (2022)](#barPlot)
- [heatMap for gene family: Angueyra et al. (2021)](#heatMap)
- [heatMap for gene list: Angueyra et al. (2021)](#heatMapList)
- [Bar plot for single gene: across studies](#barPlot_otherStudies)
- [heatMap for gene family: across studies](#heatMap_otherStudies)
- [heatMap for gene list: across studies](#heatMapList_otherStudies)
- [Bar plot for single gene: retinal cells and photoreceptor development](#barPlot_ret)
- [heatMap for gene family: retinal cells and photoreceptor development](#heatMap_ret)
- [heatMap for gene list: retinal cells and photoreceptor development](#heatMapList_ret)

***
## Bar plot for single gene: provide gene symbol (e.g. '_tbx2a_')<a id='barPlot'></a>
***
[Back to Index](#Index)

In [ ]:
geneSymbol = 'rho' # replace geneSymbol here and rerun cell to replot

barData = gf[gf['symbol']==geneSymbol] #get data (should add a check here with isin)
fH, axH = plt.subplots(figsize=(8,6))
pH = plotBars(barData, geneSymbol, ax=axH, pC=pC)
plt.subplots_adjust(left=0.15, right=.95, top=0.90, bottom=0.1)
plt.tight_layout()
plt.show()

***
## HeatMap for gene family: provide gene symbol prefix (e.g. '_opn1_')<a id='heatMap'></a>
> retrieves all genes whose symbol starts with defined geneSymbol  
***

[Back to Index](#Index)

In [ ]:
# display option for heatmap normalization
heatmapNormalization = False

In [ ]:
geneSymbol='opn' # replace geneSymbol here and rerun cell to replot

heatmapData = gf[gf['symbol'].str.startswith(geneSymbol)]
heatmapData = heatmapData.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapData.shape[0])+4))
hmH, cbH = heatmap(heatmapData, pC = pC, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

***
## HeatMap for gene list: provide gene symbols separated by "|" (e.g. '_tbx2a_|_tbx2b_|_foxq2_')<a id='heatMapList'></a>
> retrieves all genes in list and creates heatmap  
***

[Back to Index](#Index)

In [ ]:
# display option for heatmap normalization
# heatmapNorm
heatmapNormalization = True

In [ ]:
geneSymbol='tbx2a|tbx2b|foxq2|nr2e3|skor1a|sall1a|lrrfip1a' # replace geneSymbol here and rerun cell to replot

heatmapData = gf[gf['symbol'].str.contains(geneSymbol)]
heatmapData = heatmapData.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapData.shape[0])+4))
hmH, cbH = heatmap(heatmapData, pC = pC, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

***
## Bar plots for single gene across studies: provide gene symbol (e.g. '_foxq2_')<a id='barPlot_otherStudies'></a>
***
[Back to Index](#Index)

In [ ]:
geneSymbol = 'rho' # replace geneSymbol here and rerun cell to replot

barData = gf[gf['symbol']==geneSymbol] # get data
barDataO = zfO[zfO['symbol']==geneSymbol] #get data
barDataH = zfH[zfH['symbol']==geneSymbol] #get data
barDataS = zfS[zfS['symbol']==geneSymbol] #get data
fH, axH = plt.subplots(4,2,figsize=(7*2,6*4))
pH0 = plotBars(barData, geneSymbol, ax=axH[0,0], pC=pC)
plt.figtext(0.53,0.88,"Angueyra et al. (2022)", va="center", ha="center", rotation= 270, size=12)
pH1 = plotBars_Ogawa2021(barDataO, geneSymbol, axH[1,0], pC, pctPlot=False)
pH2 = plotBars_Ogawa2021(barDataO, geneSymbol, axH[1,1], pC, pctPlot=True)
plt.figtext(0.98,0.62,"Ogawa and Corbo (2021)", va="center", ha="center", rotation= 270, size=12)
pH3 = plotBars_Hoang2020(barDataH, geneSymbol, axH[2,0], pC, pctPlot=False)
pH4 = plotBars_Hoang2020(barDataH, geneSymbol, axH[2,1], pC, pctPlot=True)
plt.figtext(0.98,0.38,"Hoang et al. (2020)", va="center", ha="center", rotation= 270, size=12)
pH5 = plotBars_Sun2018(barDataS, geneSymbol, axH[3,0], pC)
axH[0,1].remove()
axH[3,1].remove()
plt.figtext(0.50,0.12,"Sun, Galicia and Stenkamp (2018)", va="center", ha="center", rotation= 270, size=12)
plt.tight_layout()
plt.show()

***
## HeatMap for gene family across studies: provide gene symbol prefix (e.g. '_tbx_')<a id='heatMap_otherStudies'></a>
> retrieves all genes whose symbol starts with defined geneSymbol  
***

[Back to Index](#Index)

In [ ]:
# display option for normalization
# heatmapNorm
heatmapNormalization = False

In [ ]:
geneSymbol='tbx' # replace geneSymbol here and rerun cell to replot

heatmapData = gf[gf['symbol'].str.startswith(geneSymbol)]
heatmapData = heatmapData.sort_values(by=["symbol"])
heatmapDataO = zfO[zfO['symbol'].str.startswith(geneSymbol)]
heatmapDataO = heatmapDataO.sort_values(by=["symbol"])
heatmapDataH = zfH[zfH['symbol'].str.startswith(geneSymbol)]
heatmapDataH = heatmapDataH.sort_values(by=["symbol"])
heatmapDataS = zfS[zfS['symbol'].str.startswith(geneSymbol)]
heatmapDataS = heatmapDataS.sort_values(by=["symbol"])


fH, axH = plt.subplots(figsize=(20,(heatmapDataO.shape[0])+4))
hm, cb = heatmap(heatmapData, pC=pC, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Angueyra et al. (2022)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()
fH, axHO = plt.subplots(figsize=(16,(heatmapDataO.shape[0])+6))
hmO, cbO = heatmap_Ogawa2021(heatmapDataO, pC=pC, pctPlot=False, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Ogawa and Corbo (2021)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()
fH, axHH = plt.subplots(figsize=(14,(heatmapDataH.shape[0])+6))
hmH, cbH = heatmap_Hoang2020(heatmapDataH, pC=pC, pctPlot=False, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Hoang et al. (2020)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()
fH, axHS = plt.subplots(figsize=(14,(heatmapDataS.shape[0])+6))
hmS, cbS = heatmap_Sun2018(heatmapDataS, pC=pC, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Sun, Galicia and Stenkamp (2018)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()


***
## HeatMap for gene list across studies: provide gene symbol separated by "|" (e.g. '_tbx2a_|_tbx2b_|_foxq2_')<a id='heatMapList_otherStudies'></a>
> retrieves all genes in list and creates heatmap  
***

[Back to Index](#Index)

In [ ]:
# display option for normalization
heatmapNormalization = False

In [ ]:
geneSymbol='tbx2a|tbx2b|foxq2' # replace geneSymbol here and rerun cell to replot

heatmapData = gf[gf['symbol'].str.contains(geneSymbol)]
heatmapData = heatmapData.sort_values(by=["symbol"])
heatmapDataO = zfO[zfO['symbol'].str.contains(geneSymbol)]
heatmapDataO = heatmapDataO.sort_values(by=["symbol"])
heatmapDataH = zfH[zfH['symbol'].str.contains(geneSymbol)]
heatmapDataH = heatmapDataH.sort_values(by=["symbol"])
heatmapDataS = zfS[zfS['symbol'].str.contains(geneSymbol)]
heatmapDataS = heatmapDataS.sort_values(by=["symbol"])

fH, axH = plt.subplots(figsize=(20,(heatmapDataO.shape[0])+4))
hm, cb = heatmap(heatmapData, pC=pC, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Angueyra et al. (2022)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()
fH, axHO = plt.subplots(figsize=(16,(heatmapDataO.shape[0])+6))
hmO, cbO = heatmap_Ogawa2021(heatmapDataO, pC=pC, pctPlot=False, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Ogawa and Corbo (2021)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()
fH, axHH = plt.subplots(figsize=(14,(heatmapDataH.shape[0])+6))
hmH, cbH = heatmap_Hoang2020(heatmapDataH, pC=pC, pctPlot=False, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Hoang et al. (2020)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()
fH, axHS = plt.subplots(figsize=(14,(heatmapDataS.shape[0])+6))
hmS, cbS = heatmap_Sun2018(heatmapDataS, pC=pC, norm=heatmapNormalization)
plt.figtext(0.98,0.015,"Sun, Galicia and Stenkamp (2018)", va="center", ha="right", rotation= 0, size=12)
plt.tight_layout()
plt.show()

***
# Development
***

***
## Bar plot for single gene for Hoang et al. (2021): retinal cells and photoreceptor development<a id='barPlot_ret'></a>
> provide gene symbol (e.g. '_tbx2a_')
***
[Back to Index](#Index)

In [ ]:
pC_Ret = {
            'RPC' : '#DADADA', # Retinal progenitor cell
            'PRPC' : '#dfdac8', # Photoreceptor progenitor cell
            'Cones_larval' : '#dcc360', #
            'Cones_adult' : '#ffd429', #
            'Rods' : '#7d7d7d', #
            'HC' : '#FC7715', # Horizontal cells
            'BC_larval' : '#ccf2ff', # Bipolar cell (developing)
            'BC_adult' : '#663d00', # Bipolar cell (mature)
            'AC_larval' : '#3DF591', # Amacrine cell (developing)
            'ACgaba' : '#3DF5C3', #
            'ACgly' : '#56F53D', #
            'RGC_larval' : '#F53D59', # Retinal Ganglion cell (developing)
            'RGC_adult' : '#BB0622', # Retinal Ganglion cell (mature)
            'MGi' : '#EA9D81', # Muller glia (immature)
            'MG1' : '#A2644E', # Muller glia (mature)
            'MG2' : '#7E4835', # Muller glia (mature)
            'MG3' : '#613728', # Muller glia (mature)
        }

pC_PhotoDev = {
    'RPC' : "#DADADA", # Retinal progenitor cell
    'PRP' : "#dfdac8", # Photoreceptor progenitor cell
    'Cle' : '#dacd9a', # Cone, larval early
    'Clm' : '#dcc360', # Cone, larval mid
    'Cll' : '#cca819', # Cone, larval late
    'C' : '#ffd429', # Cone adult
    'Rll' : '#a3a3a3', # Rod, larval late
    'R' : '#7d7d7d', # Rod adult
}

pC_42hpf = {
    'RPC' : '#DADADA', # Retinal progenitor cell
    'PR' : '#dcc360', # Photoreceptor cell (developing)
    'HC_AC' : '#3DF591', # Horizontal and Amacrine cell (developing)
    'RGC' : '#F53D59', # Retinal Ganglion cell (developing)
}

zfHRet = pd.read_csv('./data/Hoang2020_10x_retCells.csv')
print("\t Hoang et al. (2020): Retinal Cells")

zfHPhotoDev = pd.read_csv('./data/Hoang2020_10x_photoDev.csv')
print("\t Hoang et al. (2020): Photoreceptor development")

zfX = pd.read_csv('./data/Xu2020_retAll.csv')
print("\t Xu et al. (2020): retinal development")

zfN = pd.read_csv('./data/Nerli2022_42hpfRet.csv')
print("\t Nerli et al. (2022): 42hpf")

print("Successfully loaded datasets!\n\n")

In [ ]:
geneSymbol = 'rho' # replace geneSymbol here and rerun cell to replot

barDataX = zfX[zfX['symbol']==geneSymbol] #get data (should add a check here with isin)
fH, axH = plt.subplots(figsize=(25*1.5,6*1.5))
pH = plotBars_Xu2020_RetDev(barDataX, geneSymbol, ax=axH)
plt.subplots_adjust(left=0.15, right=.95, top=0.90, bottom=0.1)
plt.tight_layout()
plt.show()


barDataPhotoDev = zfHPhotoDev[zfHPhotoDev['symbol']==geneSymbol] #get data (should add a check here with isin)
fH, axH = plt.subplots(figsize=(8*1.5,6*1.5))
pH = plotBars_Hoang2020_PhotoDev(barDataPhotoDev, geneSymbol, ax=axH, pC=pC_PhotoDev)
plt.subplots_adjust(left=0.15, right=.95, top=0.90, bottom=0.1)
plt.tight_layout()
plt.show()

barDataHRet = zfHRet[zfHRet['symbol']==geneSymbol] #get data (should add a check here with isin)
fH, axH = plt.subplots(figsize=(8*1.5,6*1.5))
pH = plotBars_Hoang2020_Ret(barDataHRet, geneSymbol, ax=axH, pC=pC_Ret)
plt.subplots_adjust(left=0.15, right=.95, top=0.90, bottom=0.1)
plt.tight_layout()
plt.show()

barDataN = zfN[zfN['symbol']==geneSymbol] #get data (should add a check here with isin)
fH, axH = plt.subplots(figsize=(8*1.5,6*1.5))
pH = plotBars_Nerli2022(barDataN, geneSymbol, ax=axH, pC=pC_42hpf)
plt.subplots_adjust(left=0.15, right=.95, top=0.90, bottom=0.1)
plt.tight_layout()
plt.show()

***
## HeatMap for gene family across studies: retinal cells and photoreceptor development<a id='heatMap_ret'></a>
> retrieves all genes whose symbol starts with defined geneSymbol  
***

[Back to Index](#Index)

In [ ]:
geneSymbol='tbx2' # replace geneSymbol here and rerun cell to replot

heatmapDataX = zfX[zfX['symbol'].str.startswith(geneSymbol)]
heatmapDataX = heatmapDataX.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(60,(0.5*heatmapDataX.shape[0])+5))
hmH, cbH = heatmap_Xu2020_RetDev(heatmapDataX, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

heatmapDataHRet = zfHRet[zfHRet['symbol'].str.startswith(geneSymbol)]
heatmapDataHRet = heatmapDataHRet.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataHRet.shape[0])+8))
hmH, cbH = heatmap_Hoang2020_Ret(heatmapDataHRet, pC = pC_Ret, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

heatmapDataHPhotoDev = zfHPhotoDev[zfHPhotoDev['symbol'].str.startswith(geneSymbol)]
heatmapDataHPhotoDev = heatmapDataHPhotoDev.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataHPhotoDev.shape[0])+8))
hmH, cbH = heatmap_Hoang2020_PhotoDev(heatmapDataHPhotoDev, pC = pC_PhotoDev, norm=heatmapNormalization)
plt.tight_layout()
plt.show()


heatmapDataN = zfN[zfN['symbol'].str.startswith(geneSymbol)]
heatmapDataN = heatmapDataN.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataN.shape[0])+5))
hmH, cbH = heatmap_Nerli2022(heatmapDataN, pC = pC_42hpf, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

***
## HeatMap for gene list across studies: retinal cells and photoreceptor development<a id='heatMapList_ret'></a>
> retrieves all genes in list and creates heatmap  
> provide gene symbol separated by "|" (e.g. 'tbx2a|tbx2b|foxq2'). 
***

[Back to Index](#Index)

In [ ]:
geneSymbol='tbx2a|tbx2b|foxq2' # replace geneSymbol here and rerun cell to replot


heatmapDataX = zfX[zfX['symbol'].str.contains(geneSymbol)]
heatmapDataX = heatmapDataX.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataX.shape[0])+5))
hmH, cbH = heatmap_Xu2020_RetDev(heatmapDataN, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

heatmapDataHRet = zfHRet[zfHRet['symbol'].str.contains(geneSymbol)]
heatmapDataHRet = heatmapDataHRet.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataHRet.shape[0])+8))
hmH, cbH = heatmap_Hoang2020_Ret(heatmapDataHRet, pC = pC_Ret, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

heatmapDataHPhotoDev = zfHPhotoDev[zfHPhotoDev['symbol'].str.contains(geneSymbol)]
heatmapDataHPhotoDev = heatmapDataHPhotoDev.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataHPhotoDev.shape[0])+8))
hmH, cbH = heatmap_Hoang2020_PhotoDev(heatmapDataHPhotoDev, pC = pC_PhotoDev, norm=heatmapNormalization)
plt.tight_layout()
plt.show()


heatmapDataN = zfN[zfN['symbol'].str.contains(geneSymbol)]
heatmapDataN = heatmapDataN.sort_values(by=["symbol"])
fH, axH = plt.subplots(figsize=(20,(0.5*heatmapDataN.shape[0])+5))
hmH, cbH = heatmap_Nerli2022(heatmapDataN, pC = pC_42hpf, norm=heatmapNormalization)
plt.tight_layout()
plt.show()

# FIN